In [5]:
import json
import numpy as np
import pandas as pd

In [2]:
# csv paths
nutrient_matrix_data_p = r"../../data/Nutrients_Branded_Foods_2018/"

nutrient_matrix_csv_p = nutrient_matrix_data_p + "nutrients_matrix.csv.gz"

In [7]:
#parameters to save the linkage matrix
metric = 'euclidean'

linkage = 'ward'

fname = nutrient_matrix_data_p + 'hcluster_nutrients_' + metric + '_' + linkage + '.npy'

In [6]:
#read data
nutrients_matrix = pd.read_csv(nutrient_matrix_csv_p)
#set ndb no as index
nutrients_matrix.set_index("NDB_No", inplace = True)
#create a list of indices where all entries are zero
nutrients_zero_rows = nutrients_matrix.loc[(nutrients_matrix==0).all(axis=1)].index
#filter out indicies where all entires are zero
nutrients_matrix = nutrients_matrix[~nutrients_matrix.index.isin(nutrients_zero_rows)]

#### Create a json object from the linkage matrix

In [8]:
linkage_matrix = np.load(fname, allow_pickle=True)

Build the json object with recursion.
Inspiration:

https://stackoverflow.com/questions/65462806/how-to-parse-data-from-scikitlearn-agglomerative-clustering

In [17]:
def create_tree(linked, leaf_labels):
    ## inner func to recurvise-ly walk the linkage matrix
    def recurTree(tree):
        k = tree['clust']
        ## no children for this node
        if k not in inter:
            tree['nbd_no'] = int(leaf_labels[k])
            del tree["clust"]
            del tree["children"]
            return
        for n in inter[k]:
            ## build child nodes
            node = {
                "clust": int(n),
                "parent": k,
                "children": []
            }
            ## add to children
            tree['children'].append(node)
            ## next recursion
            recurTree(node)      
    
    num_rows, _ = linked.shape
    inter = {}
    i = 0
    # loop linked matrix convert to dictionary
    for row in linked:
        i += 1
        inter[i + num_rows] = [row[0],row[1]]

    # start tree data structure
    tree = {
        "clust": i + num_rows,
        #"parent": None,
        "children": []
    }
    # start recursion
    recurTree(tree);
    return tree

In [ ]:
d = create_tree(linkage_matrix, nutrients_matrix.index)

In [41]:
json.dump(d, open(nutrient_matrix_data_p + 'hcluster_nutrients_' + metric + '_' + linkage + 'json', "w"))

Function version of building the json object without recursion

In [6]:
def test(linked, leaf_labels):
    num_rows, _ = linked.shape
    inter = {}
    i = 0
    # loop linked matrix convert to dictionary
    for row in linked:
        #print(row[0], row[1])
        i += 1
        #if the index is within the original matrix, then its a leaf node
        if row[0] <= num_rows:
            child1 = { "nbd_no": int(leaf_labels[row[0]]), "parent":  int(i + num_rows)}
        else:
            child1 = {"cluster": int(row[0]), "parent": int(i + num_rows), "children": inter[row[0]]['children']}
            del inter[row[0]]
        if row[1] <= num_rows:
            child2 = { "nbd_no": int(leaf_labels[row[1]]), "parent":  int(i + num_rows)}
        else:
            child2 = {"cluster": int(row[1]), "parent": int(i + num_rows), "children": inter[row[1]]['children']}    
            del inter[row[1]]
        inter[i + num_rows] = {"cluster": int(i + num_rows),
            #"parent": None,
            "children": [child1, child2]
                              }
        #print(inter)
    return inter[i + num_rows]

In [11]:
g = test(linkage_matrix, nutrients_matrix.index)

In [ ]:
g

In [ ]:
json.dump(g, open('tst.json', "w"))